# Bioclimatic variables

Compute the bioclimatic variables as defined in https://pubs.usgs.gov/ds/691/ds691.pdf.

In [4]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from os.path import isfile
import zarr

In [5]:
dsspat = xr.open_zarr("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_spatial.zarr/")
dstime = xr.open_zarr("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_temporal.zarr/")
subset = dstime.isel(Time=range(0, 2*365+1)) #two year subset for faster testing

/home/zhang/miniconda3/envs/data_cube2/lib/python3.10/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'hurs' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/zhang/miniconda3/envs/data_cube2/lib/python3.10/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/zhang/miniconda3/envs/data_cube2/lib/python3.10/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'rsds' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/zhang/miniconda3/envs/data_cube2/lib/python3.10/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/zhang/miniconda3/envs/data_cube2/lib/python3.

In [19]:
dsspat

<xarray.Dataset>
Dimensions:                     (Time: 8766, rlat: 824, rlon: 848, vertices: 4)
Coordinates:
  * Time                        (Time) datetime64[ns] 1995-01-01 ... 2018-12-31
    height                      float64 ...
    latitude                    (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
    longitude                   (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * rlat                        (rlat) float64 -23.4 -23.35 ... 21.81 21.86
  * rlon                        (rlon) float64 -28.4 -28.35 ... 18.13 18.18
  * vertices                    (vertices) float64 1.0 2.0 3.0 4.0
Data variables:
    hurs                        (Time, rlat, rlon) float32 dask.array<chunksize=(5, 824, 848), meta=np.ndarray>
    pr                          (Time, rlat, rlon) float32 dask.array<chunksize=(5, 824, 848), meta=np.ndarray>
    rotated_latitude_longitude  float64 ...
    rsds                        (Time, rlat, rlon) float32 dask.array<chunksize=(5, 824, 848), meta=np.ndarray>
    tas                         (Time, rlat, rlon) float32 dask.array<chunksize=(5, 824, 848), meta=np.ndarray>
    vertices_latitude           (rlat, rlon, vertices) float64 dask.array<chunksize=(412, 424, 1), meta=np.ndarray>
    vertices_longitude          (rlat, rlon, vertices) float64 dask.array<chunksize=(412, 424, 1), meta=np.ndarray>
Attributes: (12/45)
    CDO:                            Climate Data Operators version 2.2.1 (htt...
    CORDEX_domain:                  EUR-6km
    Conventions:                    CF-1.7
    activity_id:                    COSMO-REA
    cmor_version:                   3.6.0
    contact:                        cdc.rea@dwd.de
    ...                             ...
    table_info:                     Creation Date:(27 Jan 2023) MD5:509fc5c33...
    title:                          COSMO-REA6 regional reanalysis data prepa...
    tracking_id:                    hdl:21.14103/2d8fab63-e5ec-4fef-838b-0ba9...
    variable_id:                    tas
    variant_info:                   Regional reanalysis forced by ERA-Interim.
    variant_label:                  r1i1p1f1

In [11]:
za = zarr.open("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_spatial.zarr/", mode='a')

# Intermediate variables

### Temperature

In [10]:
# Monthly max:
if isfile("./datasets/tas_mmax.nc"):
    tas_mmax = xr.open_dataarray("./datasets/tas_mmax.nc")
else:
    tas_mmax = dstime["tas"].resample(Time="1MS").max(dim="Time")
    tas_mmax.to_netcdf("./datasets/tas_mmax.nc")
    
    
# Monthly min:
if isfile("./datasets/tas_mmin.nc"):
    tas_mmin = xr.open_dataarray("./datasets/tas_mmin.nc")
else:
    tas_mmin = dstime["tas"].resample(Time="1MS").min(dim="Time")
    tas_mmin.to_netcdf("./datasets/tas_mmin.nc")
    
    
# Monthly mean:
if isfile("./datasets/tas_mmean.nc"):
    tas_mmean = xr.open_dataarray("./datasets/tas_mmean.nc")
else:
    tas_mmean = dstime["tas"].resample(Time="1MS").mean(dim="Time")
    tas_mmean.to_netcdf("./datasets/tas_mmean.nc")

### Precipitation

In [32]:
# Monthly total:
if isfile("./datasets/prec_mtot.nc"):
    prec_mtot = xr.open_dataarray("./datasets/prec_mtot.nc")
else:
    prec_mtot = dstime["pr"].resample(Time="1MS").sum(dim="Time") * 86400 #convert from flux to total in mm
    prec_mtot = prec_mtot.assign_attrs({"standard_name": "total_precipitation", "units": "mm"})
    prec_mtot.to_netcdf("./datasets/prec_mtot.nc")

# [BIO1] Annual Mean Temperature

In [33]:
ds_bio1 = dstime["tas"].groupby("Time.year").mean("Time")
ds_bio1.to_netcdf("./datasets/bio1.nc")

# [BIO2] Mean Diurnal Range

The mean diurnal range is defined as mean of monthly maximum temperatures and minimum temperatures, i.e.: MDR $= \frac{1}{12} \sum_{i=1}^{12} T_\text{max} - T_\text{min}$

In [34]:
monthly_range = tas_mmax - tas_mmin

ds_bio2 = monthly_range.groupby("Time.year").mean(dim="Time")
ds_bio2.to_netcdf("./datasets/bio2.nc")

# [BIO4] Temperature Seasonality

Temperature seasonality is defined as the standard deviation (over one year) of monthly temperature averages.

In [35]:
ds_bio4 = tas_mmean.groupby("Time.year").std(dim="Time")
ds_bio4.to_netcdf("./datasets/bio4.nc")

# [BIO5] Max Temperature of Warmest Month

In [36]:
ds_bio5 = tas_mmax.groupby("Time.year").max(dim="Time")
ds_bio5.to_netcdf("./datasets/bio5.nc")

# [BIO6] Min Temperature of Coldest Month

In [37]:
ds_bio6 = tas_mmin.groupby("Time.year").min(dim="Time")
ds_bio6.to_netcdf("./datasets/bio6.nc")

# [BIO7] Temperature Annual Range

Defined as: $\text{BIO5} - \text{BIO6}$

In [38]:
ds_bio7 = ds_bio5 - ds_bio6
ds_bio7.to_netcdf("./datasets/bio7.nc")

# [BIO3] Isothermality

Defined as: $\frac{\text{BIO2}}{\text{BIO7}} \cdot 100$\
Execute below cell only after computing BIO7...

In [39]:
ds_bio3 = ds_bio2 / ds_bio7 * 100.
ds_bio3.to_netcdf("./datasets/bio3.nc")

# [BIO8/9] Mean Temperature of Wettest/Driest Quarter

In our [reference](https://pubs.usgs.gov/ds/691/ds691.pdf), the quartals are defined as 12 rolling three month windows looking forward, i.e., Jan-Feb-Mar, Feb-Mar-Apr, ..., Okt-Nov-Dec, Nov-Dec-Jan, Dec-Jan-Feb (12 in total). Annoyingly, xarrays rolling functionality only does backwards or centered rolling windows, which is why we have to loop here.

In [57]:
ds_bio8 = []
ds_bio9 = []

for year in set(dstime["Time.year"].values):
    tas_quarterly = [] #mean
    precip_quarterly = [] #total
    
    # For each year loop through all the quarters as defined above:
    start_dates = np.arange(np.datetime64(f"{year}-01"), np.datetime64(f"{year+1}-01"), np.timedelta64(1, "M"))
    for start_date in start_dates:
        slice_quart = slice(start_date, start_date + np.timedelta64(2, "M")) #quarter as time slice
        
        tas_quarterly.append(tas_mmean.sel(Time=slice_quart).mean(dim="Time")) #mean temperature per grid point
        precip_quarterly.append(prec_mtot.sel(Time=slice_quart).sum(dim="Time")) #total precipitation per grid point
    
    # Get indices of wettest quarters this year:
    precip_quarterly = xr.concat(precip_quarterly, dim="Quarter")
    wettest_quarters = precip_quarterly.argmax(dim="Quarter")
    driest_quarters = precip_quarterly.argmin(dim="Quarter")
    
    # Get mean temperatures in wettest/driest quarters:
    tas_quarterly = xr.concat(tas_quarterly, dim="Quarter")
    ds_bio8.append(tas_quarterly.compute().isel(Quarter=wettest_quarters.compute()))
    ds_bio9.append(tas_quarterly.compute().isel(Quarter=driest_quarters.compute()))
    
# Save:
ds_bio8 = xr.concat(ds_bio8, dim="year")
ds_bio8.to_netcdf("./datasets/bio8.nc")

ds_bio9 = xr.concat(ds_bio9, dim="year")
ds_bio9.to_netcdf("./datasets/bio9.nc")

# [BIO10/11] Mean Temperature of Warmest/Coldest Quarter

In [58]:
ds_bio10 = []
ds_bio11 = []

for year in set(dstime["Time.year"].values):
    tas_quarterly = [] #mean
    
    # For each year loop through all the quarters as defined in description of BIO8:
    start_dates = np.arange(np.datetime64(f"{year}-01"), np.datetime64(f"{year+1}-01"), np.timedelta64(1, "M"))
    for start_date in start_dates:
        slice_quart = slice(start_date, start_date + np.timedelta64(2, "M")) #quarter as time slice
        
        tas_quarterly.append(tas_mmean.sel(Time=slice_quart).mean(dim="Time")) #mean temperature per grid point
        
    ds_bio10.append(xr.concat(tas_quarterly, dim="Quarter").max(dim="Quarter"))
    ds_bio11.append(xr.concat(tas_quarterly, dim="Quarter").min(dim="Quarter"))
    
    
# Save:
ds_bio10 = xr.concat(ds_bio10, dim="year")
ds_bio10.to_netcdf("./datasets/bio10.nc")

ds_bio11 = xr.concat(ds_bio11, dim="year")
ds_bio11.to_netcdf("./datasets/bio11.nc")

# [BIO12] Annual Precipitation

In [59]:
ds_bio12 = prec_mtot.groupby("Time.year").sum(dim="Time")
ds_bio12.to_netcdf("./datasets/bio12.nc")

# [BIO13] Precipitation of Wettest Month

In [60]:
ds_bio13 = prec_mtot.groupby("Time.year").max(dim="Time")
ds_bio13.to_netcdf("./datasets/bio13.nc")

# [BIO14] Precipitation of Driest Month

In [61]:
ds_bio14 = prec_mtot.groupby("Time.year").min(dim="Time")
ds_bio14.to_netcdf("./datasets/bio14.nc")

# [BIO15] Precipitation Seasonality

Precipitation seasonality in a given year is defined as: 
$$
\frac{\sigma(P_i)}{1 + \text{BIO12}\,/\,12} \cdot 100 \quad \text{for}\, i \in [1,12]
$$

In [62]:
ds_bio15 = prec_mtot.groupby("Time.year").std(dim="Time") / (1. + ds_bio12/12.) * 100
ds_bio15.to_netcdf("./datasets/bio15.nc")

# [BIO16/17] Precipitation of Wettest/Driest Quarter

In [63]:
ds_bio16 = []
ds_bio17 = []

for year in set(dstime["Time.year"].values):
    precip_quarterly = [] #mean
    
    # For each year loop through all the quarters as defined in description of BIO8:
    start_dates = np.arange(np.datetime64(f"{year}-01"), np.datetime64(f"{year+1}-01"), np.timedelta64(1, "M"))
    for start_date in start_dates:
        slice_quart = slice(start_date, start_date + np.timedelta64(2, "M")) #quarter as time slice
        
        precip_quarterly.append(prec_mtot.sel(Time=slice_quart).sum(dim="Time")) #total precipitation per grid point
        
    ds_bio16.append(xr.concat(precip_quarterly, dim="Quarter").max(dim="Quarter"))
    ds_bio17.append(xr.concat(precip_quarterly, dim="Quarter").min(dim="Quarter"))

    
# Save:
ds_bio16 = xr.concat(ds_bio16, dim="year")
ds_bio16.to_netcdf("./datasets/bio16.nc")

ds_bio17 = xr.concat(ds_bio17, dim="year")
ds_bio17.to_netcdf("./datasets/bio17.nc")

# [BIO18/19] Precipitation of Warmest/Coldest Quarter

In [65]:
ds_bio18 = []
ds_bio19 = []

for year in set(dstime["Time.year"].values):
    tas_quarterly = [] #mean
    precip_quarterly = [] #total
    
    # For each year loop through all the quarters as defined above:
    start_dates = np.arange(np.datetime64(f"{year}-01"), np.datetime64(f"{year+1}-01"), np.timedelta64(1, "M"))
    for start_date in start_dates:
        slice_quart = slice(start_date, start_date + np.timedelta64(2, "M")) #quarter as time slice
        
        tas_quarterly.append(tas_mmean.sel(Time=slice_quart).mean(dim="Time")) #mean temperature per grid point
        precip_quarterly.append(prec_mtot.sel(Time=slice_quart).sum(dim="Time")) #total precipitation per grid point
    
    # Get indices of warmest quarters this year:
    tas_quarterly = xr.concat(tas_quarterly, dim="Quarter")
    warmest_quarters = tas_quarterly.argmax(dim="Quarter")
    coldest_quarters = tas_quarterly.argmin(dim="Quarter")
    
    # Get precipitation in warmest quarters:
    precip_quarterly = xr.concat(precip_quarterly, dim="Quarter")
    ds_bio18.append(precip_quarterly.compute().isel(Quarter=warmest_quarters.compute()))
    ds_bio19.append(precip_quarterly.compute().isel(Quarter=coldest_quarters.compute()))
    
    
# Save:
ds_bio18 = xr.concat(ds_bio18, dim="year")
ds_bio18.to_netcdf("./datasets/bio18.nc")

ds_bio19 = xr.concat(ds_bio19, dim="year")
ds_bio19.to_netcdf("./datasets/bio19.nc")